In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
import re
import os.path
from os import path
from definitions import *

In [ ]:
def compute_P(I_up,I_down, dI_up, dI_down):
    P = (I_up - I_down) / (I_up + I_down)
    dP = 2 * np.sqrt((I_down * dI_up)**2 + (I_up * dI_down)**2) / (I_up + I_down) ** 2 
    return P, dP

def compute_I_diff(I_up,I_down, dI_up, dI_down):
    I_diff = (I_up - I_down)
    dI_diff = np.sqrt(dI_up ** 2 + dI_down ** 2)
    return I_diff, dI_diff

def compute_rms(x, dx):
    rms = np.sqrt(np.mean(np.square(x)))
    squared_errors = np.square(dx)
    mean_squared_values = np.mean(np.square(x))
    d_rms = np.sqrt(np.sum(squared_errors) / (2 * len(x) * mean_squared_values))
    return rms, d_rms 

def get_measurement_point(i, folder='data'):
    d1 = np.genfromtxt(f'{folder}/up/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    d2 = np.genfromtxt(f'{folder}/down/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    d3 = np.genfromtxt(f'{folder}/empty_up/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    d4 = np.genfromtxt(f'{folder}/empty_down/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    up = DetectorReading(*d1)
    down = DetectorReading(*d2)
    empty_up = DetectorReading(*d3)
    empty_down = DetectorReading(*d4)
    parameters = extract_parameters(f'{folder}/up/{i}/det.dat')
    mp = MeasurementPoint(up, down, empty_up, empty_down, parameters)
    return mp
    

class DetectorReading:
    def __init__(self, y, I, I_err, N):
        self.y = y * 10
        self.I = I
        self.dI = I_err
        self.N = N

class MeasurementPoint:
    def __init__(self, up, down, empty_up, empty_down, parameters):
        self.up = up
        self.down = down
        self.empty_up = empty_up
        self.empty_down = empty_down
        self.parameters = parameters
    
    def y(self):
        return self.up.y
    
    # Computes just I_up - I_down
    def compute_I_diffs(self):
        I_diff_s, dI_diff_s = compute_I_diff(self.up.I, self.down.I, self.up.dI, self.down.dI)
        I_diff_b, dI_diff_b = compute_I_diff(self.empty_up.I, self.empty_down.I, self.empty_up.dI, self.empty_down.dI)
        return self.y(), I_diff_b, dI_diff_b, I_diff_s, dI_diff_s

    def compute_P(self):
        P_s, dP_s = compute_P(self.up.I, self.down.I, self.up.dI, self.down.dI)
        P_b, dP_b = compute_P(self.empty_up.I, self.empty_down.I, self.empty_up.dI, self.empty_down.dI)
        return self.y(), P_b, dP_b, P_s, dP_s
    
    def compute_rms(self, y_range = None):
        y, P_b, dP_b, P_s, dP_s = self.compute_P()
        # y, P_b, dP_b, P_s, dP_s = self.compute_I_diffs()
        if y_range == None:
            rms_b, d_rms_b = compute_rms(P_b, dP_b)
            rms_s, d_rms_s = compute_rms(P_s, dP_s)
        else:
            # range is assumed to be of the form (y_min, y_max)
            indices = indices_within_range(y, *y_range)
            rms_b, d_rms_b = compute_rms(P_b[indices], dP_b[indices])
            rms_s, d_rms_s = compute_rms(P_s[indices], dP_s[indices])
        return rms_b, d_rms_b, rms_s, d_rms_s

    def rms_ratio(self, y_range = None):
        rms_b, d_rms_b, rms_s, d_rms_s = self.compute_rms(y_range=y_range)
        R = rms_s / rms_b
        dR = np.sqrt((d_rms_s / rms_b)**2 + (rms_s * d_rms_b / rms_b**2)**2)
        return R, dR
    
    def plot_intensities(self):
        plt.plot(self.y(), self.empty_up.I, label='Up')
        plt.plot(self.y(), self.empty_down.I, label = 'Down')
        plt.title(r'Base $I_{up}, I_{down}$')
        plt.grid()
        plt.legend()
        plt.show()    
        plt.plot(self.y(), self.up.I, label='Up')
        plt.plot(self.y(), self.down.I, label = 'Down')
        plt.title(r'Sample $I_{up}, I_{down}$')
        plt.grid()
        plt.legend()
        plt.show()


In [ ]:
x = get_measurement_point(4, 'foil_data')
y, P_b, dP_b, P_s, dP_s = x.compute_P()
y_range=(-4.5,4.5)
rms_b, d_rms_b, rms_s, d_rms_s = x.compute_rms(y_range)
print(rms_b, d_rms_b, rms_s, d_rms_s)
R, dR = x.rms_ratio(y_range)
print(R, dR)
# print(P, dP)
plt.plot(y,P_b)
plt.plot(y,P_s)
plt.show()
x.plot_intensities()
# plt.errorbar(y,P, linestyle='',yerr=dP, ecolor='red')
# plt.plot(y, dP)

In [ ]:
def fit_gauss(y, I, dI, y_range, By, lambda_0, theta_0, wl_sigma):   
    indices = indices_within_range(y * 1e-3, *y_range)
    def I_fit(y, A_0, I_0):
        return I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=wl_sigma) * A_0 + I_0
    # Todo: use dI for better estimate?
    popt, pcov = curve_fit(I_fit, y[indices], I[indices], absolute_sigma=False)
    # print(popt, pcov)
    A_0 = popt[0]
    dA_0 = np.sqrt(pcov[0][0])
    I_0 = popt[1]
    dI_0 = np.sqrt(pcov[1][1])
    # print(I_0, dI_0)
    return A_0, dA_0, I_0, dI_0


y, I_diff_b, dI_diff_b, I_diff_s, dI_diff_s = x.compute_I_diffs()
y_range = (np.min(y), np.max(y))
y_fit = np.linspace(*y_range, 1000)
lambda_0 = 2.165e-10
theta_0 = 0.095993
By = float(x.parameters['By'])
print(I_diff_b, I_diff_s)
A_0_s, dA_0_s, I_0_s, dI_0_s = fit_gauss(y, I_diff_s, dI_diff_s, y_range, By, lambda_0, theta_0, wl_sigma=0.1e-10)
A_0_b, dA_0_b, I_0_s, dI_0_s = fit_gauss(y, I_diff_b, dI_diff_b, y_range, By, lambda_0, theta_0, wl_sigma=0.1e-10)
fitted_s = A_0_s * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=0.1e-10)
fitted_b = A_0_b * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=0.1e-10)
# plt.plot(y,I_diff_b)

plt.plot(y,I_diff_s)
plt.plot(y, fitted_s)

In [ ]:
plt.errorbar(y, I_diff_b, linestyle=' ', yerr = dI_diff_b, ecolor='red')
plt.plot(y, fitted_b)

In [ ]:
plt.plot(y, dI_diff_b)
plt.plot(y, x.empty_up.dI)
plt.plot(y, x.empty_down.dI)
plt.show()
plt.plot(y, dI_diff_s)
plt.plot(y, x.up.dI)
plt.plot(y, x.down.dI)

In [ ]:
y_range = (-4,4)
A_0_s, dA_0_s, I_0_s, dI_0_s = fit_gauss(y, I_diff_s, dI_diff_s, y_range, By, lambda_0, theta_0, wl_sigma=0.1e-10)
A_0_b, dA_0_b, I_0_b, dI_0_b = fit_gauss(y, I_diff_b, dI_diff_b, y_range, By, lambda_0, theta_0, wl_sigma=0.1e-10)
fitted_s = A_0_s * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=0.1e-10)
fitted_b = A_0_b * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=0.1e-10)
plt.plot(y,I_diff_b, '.')
plt.plot(y,I_diff_s, '.')
plt.plot(y, fitted_s)
plt.plot(y, fitted_s)

# Gaussian envelope fitting 

In [ ]:
from instrument import *
from util import *
intrs = load_instruments('simulations_new.csv')
def clean_data(array):
    # Use np.isnan to replace NaN values with zero
    array[np.isnan(array)] = 0
    # Use np.isinf to replace positive and negative infinity with zero
    array[np.isinf(array)] = 0
    return array

number_mapping = {
    '1': '2',
    '2': '4.321',
    '3': '8'
}

letter_mapping = {
    'a': 'FOIL',
    'b': 'WSP',
    'c': 'ISO'
}

def replace_string(input_string):
    # Extract the number and letter from the input string
    number = input_string[0]
    letter = input_string[1]
    
    # Replace the number and letter according to the mappings
    replaced_number = number_mapping.get(number, number)
    replaced_letter = letter_mapping.get(letter, letter)
    
    # Return the concatenated result
    return replaced_letter + ' ' + replaced_number

for instr in intrs[3:-6]:
    for radius in [500,3000, 20000]:
# for radius in [3000]:
    # print(f"SAMPLE R = {radius*0.1}nm")
    # for instr in intrs[3:4]:
        N = 30
        B = np.zeros(N)
        delta = np.zeros(N)
        P = np.zeros(N)
        dP = np.zeros(N)
        for i in range(N):
            measurement = get_measurement_point(i, f'data_{instr.id}_{radius}')
            y_range=(-4.0e-3,4.0e-3)
            R, dR = measurement.rms_ratio(y_range)
            # y, P_b, dP_b, P_s, dP_s = measurement.compute_P()
            y, I_diff_b, dI_diff_b, I_diff_s, dI_diff_s = measurement.compute_I_diffs()
            By = float(measurement.parameters['By'])
            lambda_0 = float(measurement.parameters['L0']) * 1e-10
            lambda_sigma = float(measurement.parameters['DL']) * 1e-10
            theta_0 = float(measurement.parameters['theta0'])
            B[i] = By
            A_0_s, dA_0_s, I_0_s, dI_0_s = fit_gauss(y, I_diff_s, dI_diff_s, y_range, By, lambda_0, theta_0, lambda_sigma)
            A_0_b, dA_0_b, I_0_b, dI_0_b = fit_gauss(y, I_diff_b, dI_diff_b, y_range, By, lambda_0, theta_0, lambda_sigma)
            # A_0_s, dA_0_s = fit_gauss(y, clean_data(P_s), y_range, By, lambda_0, theta_0, lambda_sigma)
            # A_0_b, dA_0_b = fit_gauss(y, clean_data(P_b), y_range, By, lambda_0, theta_0, lambda_sigma)
            # fitted_s = A_0_s * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, lambda_sigma)
            # fitted_b = A_0_b * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, lambda_sigma)
            # print(R, dR)
            R = A_0_s / A_0_b
            dR = np.sqrt((dA_0_s / A_0_b)**2 + (A_0_s * dA_0_b / A_0_b**2)**2)
            P[i] = R
            dP[i] = dR

            delta[i] = compute_z(B[i],instr.theta_0,instr.L0,instr.L_s)

        min_delta, max_delta = np.min(delta), np.max(delta)
        t = float(measurement.parameters['t'])
        R = float(measurement.parameters['R']) * 1e-10
        delta_range = np.linspace(0, R * 3, 1000)
        wavelength = float(measurement.parameters['L0']) * 1e-10
        print(t,R,wavelength)
        # def P_analytical(z, R, t, wavelength):

        P_an = P_analytical(delta_range, R, t, wavelength)
        name = instr.name
        # plt.title(rf"$P(\delta)$ for instrument {name}")
        # plt.plot(delta * 1e9,P, '.',label=rf'$R = {round(R*1e9)}$nm, fit')
        plt.errorbar(delta * 1e9,P, ms=3,fmt='o', linestyle=' ', yerr=dP,label=rf'$R = {round(R*1e9)}$nm, fit')
        plt.plot(delta_range * 1e9, P_an, lw = 1.5, label=rf'$R = {round(R*1e9)}$nm, analytical')
    # plt.ylim((0,1))
    plt.xscale('log')
    plt.xlim((10, R * 3 * 1e9))
    
    plt.xlabel(r'$\delta$(nm)')
    plt.ylabel(r'$P(\delta)$')
    plt.legend()
    plt.grid()
    plt.savefig(f"docs/simulation-plot-{name.replace(' ','-')}.eps",bbox_inches="tight", pad_inches=0, format='eps')
    plt.show()

# RMS based fitting

In [ ]:
from instrument import *
from util import *
intrs = load_instruments('simulations_new.csv')

radii = [500,3000, 20000] # Å

for instr in intrs[3:-6]:
    for radius in radii:
# for radius in [3000]:
        print(f"SAMPLE R = {radius*0.1}nm")
    # for instr in intrs[3:-8]:
        N = 30
        B = np.zeros(N)
        delta = np.zeros(N)
        P = np.zeros(N)
        dP = np.zeros(N)
        for i in range(N):
            measurement = get_measurement_point(i, f'data_{instr.id}_{radius}')
            y_range=(-4e-3,4e-3)
            R, dR = measurement.rms_ratio(y_range)
            # print(R, dR)
            P[i] = R
            dP[i] = dR
            B[i] = float(measurement.parameters['By'])
            delta[i] = compute_z(B[i],instr.theta_0,instr.L0,instr.L_s)
        # plt.plot(delta * 1e9,P, '.',label=f'{instr.id}')
        min_delta, max_delta = np.min(delta), np.max(delta)
        delta_range = np.linspace(0, max_delta, 1000)
        t = float(measurement.parameters['t'])
        R = float(measurement.parameters['R']) * 1e-10
        wavelength = float(measurement.parameters['L0']) * 1e-10
        # def P_analytical(z, R, t, wavelength):

        P_an = P_analytical(delta_range, R, t, wavelength)
        plt.errorbar(delta * 1e9,P, ms=3,fmt='o', linestyle=' ', yerr=dP,label=rf'$R = {round(R*1e9)}$nm, fit')
        plt.plot(delta_range * 1e9, P_an, lw = 1.5, label=rf'$R = {round(R*1e9)}$nm, analytical')
    plt.xscale('log')
    plt.xlim((10, R * 3 * 1e9))
    plt.grid()       
    plt.xlabel(r'$\delta$(nm)')
    plt.ylabel(r'$P(\delta)$')
    # plt.ylim((0,1))
    plt.legend()
    plt.show()

In [ ]:
intrs = load_instruments('simulations_new.csv')

for radius in [500,3000, 20000]:
# for radius in [3000]:
    print(f"SAMPLE R = {radius*0.1}nm")
    for instr in intrs[3:-6]:
        if instr.id != '2a' or radius != 500:
            continue
    # for instr in intrs[3:-8]:
        N = 30
        B = np.zeros(N)
        delta = np.zeros(N)
        P = np.zeros(N)
        dP = np.zeros(N)
        for i in range(3):
            measurement = get_measurement_point(i, f'data_{instr.id}_{radius}')
            y_range=(-3.5,3.5)
            R, dR = measurement.rms_ratio(y_range)
            # print(R, dR)
            P[i] = R
            dP[i] = dR
            B[i] = float(measurement.parameters['By'])
            delta[i] = compute_z(B[i],instr.theta_0,instr.L0,instr.L_s)
            measurement.plot_intensities()
            
        plt.plot(delta * 1e9,P, '.',label=f'{instr.id}')
        min_delta, max_delta = np.min(delta), np.max(delta)
        delta_range = np.linspace(0, max_delta, 1000)
        t = float(measurement.parameters['t'])
        R = float(measurement.parameters['R']) * 1e-10
        wavelength = float(measurement.parameters['L0']) * 1e-10
        # def P_analytical(z, R, t, wavelength):

        P_an = P_analytical(delta_range, R, t, wavelength)
        plt.errorbar(delta * 1e9,P, linestyle=' ', yerr=dP)
        plt.plot(delta_range * 1e9, P_an, label='Analytical')
        # For my sanity, I will revert to assuming By is also the difference again as I did before
        
        plt.xlabel(r'$\delta$(nm)')
        plt.ylim((0,1))
        plt.legend()
        plt.show()

In [ ]:
from instrument import *
from util import *
intrs = load_instruments('simulations_new.csv')
def clean_data(array):
    # Use np.isnan to replace NaN values with zero
    array[np.isnan(array)] = 0
    # Use np.isinf to replace positive and negative infinity with zero
    array[np.isinf(array)] = 0
    return array

for instr in intrs[3:-6]:
    for radius in [500,3000, 20000]:
# for radius in [3000]:
    # print(f"SAMPLE R = {radius*0.1}nm")
    # for instr in intrs[3:4]:
        N = 30
        B = np.zeros(N)
        delta = np.zeros(N)
        P = np.zeros(N)
        dP = np.zeros(N)
        for i in range(N):
            measurement = get_measurement_point(i, f'data_{instr.id}_{radius}')
            y_range=(-4.5,4.5)
            R, dR = measurement.rms_ratio(y_range)
            # y, P_b, dP_b, P_s, dP_s = measurement.compute_P()
            y, I_diff_b, dI_diff_b, I_diff_s, dI_diff_s = measurement.compute_I_diffs()
            By = float(measurement.parameters['By'])
            lambda_0 = float(measurement.parameters['L0']) * 1e-10
            lambda_sigma = float(measurement.parameters['DL']) * 1e-10
            theta_0 = float(measurement.parameters['theta0'])
            B[i] = By
            A_0_s, dA_0_s, I_0_s, dI_0_s = fit_gauss(y, I_diff_s, dI_diff_s, y_range, By, lambda_0, theta_0, lambda_sigma)
            A_0_b, dA_0_b, I_0_b, dI_0_b = fit_gauss(y, I_diff_b, dI_diff_b, y_range, By, lambda_0, theta_0, lambda_sigma)
            # A_0_s, dA_0_s = fit_gauss(y, clean_data(P_s), y_range, By, lambda_0, theta_0, lambda_sigma)
            # A_0_b, dA_0_b = fit_gauss(y, clean_data(P_b), y_range, By, lambda_0, theta_0, lambda_sigma)
            # fitted_s = A_0_s * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, lambda_sigma)
            # fitted_b = A_0_b * I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, lambda_sigma)
            # print(R, dR)
            R = A_0_s / A_0_b
            dR = np.sqrt((dA_0_s / A_0_b)**2 + (A_0_s * dA_0_b / A_0_b**2)**2)
            P[i] = R
            dP[i] = dR

            delta[i] = compute_z(B[i],instr.theta_0,instr.L0,instr.L_s)

        min_delta, max_delta = np.min(delta), np.max(delta)
        t = float(measurement.parameters['t'])
        R = float(measurement.parameters['R']) * 1e-10
        delta_range = np.linspace(0, R * 3, 1000)
        wavelength = float(measurement.parameters['L0']) * 1e-10
        print(t,R,wavelength)
        # def P_analytical(z, R, t, wavelength):

        P_an = P_analytical(delta_range, R, t, wavelength)
        name = replace_string(instr.id)
        plt.title(rf"$P(\delta)$ for instrument {name}")
        # plt.plot(delta * 1e9,P, '.',label=rf'$R = {round(R*1e9)}$nm, fit')
        plt.errorbar(delta * 1e9,P, ms=3,fmt='o', linestyle=' ', yerr=dP,label=rf'$R = {round(R*1e9)}$nm, fit')
        plt.plot(delta_range * 1e9, P_an, lw = 1.5, label=rf'$R = {round(R*1e9)}$nm, analytical')
        # For my sanity, I will revert to assuming By is also the difference again as I did before
        # plt.ylim((0,1))
        plt.xscale('log')
        plt.xlim((10, R * 3 * 1e9))
        
        plt.xlabel(r'$\delta$(nm)')
        plt.ylabel(r'$P(\delta)$')
        plt.legend()
        plt.grid()
        plt.savefig(f"docs/simulation-plot-{name.replace(' ','-')}.eps",bbox_inches="tight", pad_inches=0, format='eps')
        plt.show()